In [29]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
from urllib.parse import quote_plus
from IPython.display import display, HTML
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

username = quote_plus("balamariappan20")
password = quote_plus("Naruto@2024")
clustername = "airbnb.hrkwsrt.mongodb.net"
database_name = "sample_airbnb"
connection_string = f"mongodb+srv://{username}:{password}@{clustername}/?retryWrites=true&w=majority&appName=airbnb"
client = MongoClient(connection_string)
db = client['sample_airbnb']
collection = db['listingsAndReviews']

In [157]:
airbnb_data = list(collection.find({}, {"_id": 1, "listing_url": 1, "name": 1, 'property_type': 1, 'room_type': 1, 'bed_type': 1, 'minimum_nights': 1, 'maximum_nights': 1, 'cancellation_policy': 1, 'first_review':1 , 'last_review':1,'accommodates': 1, 'bedrooms': 1, 'beds': 1, 'number_of_reviews': 1, 'price': 1, 'security_deposit':1, 'cleaning_fee': 1, 'guests_included': 1, 'images.picture_url': 1, 'review_scores': 1}))
airbnb_data = pd.DataFrame(airbnb_data)

In [158]:
airbnb_data.dropna(subset=['first_review', 'last_review'], inplace=True)

# Convert 'first_review' and 'last_review' columns to datetime
airbnb_data['first_review'] = pd.to_datetime(airbnb_data['first_review'])
airbnb_data['last_review'] = pd.to_datetime(airbnb_data['last_review'])

# Convert datetime to string with date format
airbnb_data['first_review'] = airbnb_data['first_review'].dt.strftime('%d-%m-%Y')
airbnb_data['last_review'] = airbnb_data['last_review'].dt.strftime('%d-%m-%Y')

In [159]:
airbnb_data["review_scores"] = airbnb_data["review_scores"].apply(lambda x: x.get("review_scores_rating", 0) if isinstance(x, dict) else 0)
airbnb_data.describe().T

,count,mean,std,min,25%,50%,75%,max
accommodates,4167.0,3.554356,2.246106,1.0,2.0,3.0,4.0,16.0
bedrooms,4164.0,1.396254,1.029727,0.0,1.0,1.0,2.0,20.0
beds,4165.0,2.073469,1.605819,0.0,1.0,2.0,3.0,25.0
number_of_reviews,4167.0,36.802016,54.475793,1.0,3.0,14.0,47.0,533.0
review_scores,4167.0,91.177826,15.967932,0.0,90.0,95.0,99.0,100.0


In [160]:
airbnb_data["beds"].fillna(0,inplace= True)
airbnb_data["bedrooms"].fillna(0,inplace= True)
airbnb_data["security_deposit"].fillna(0,inplace= True)
airbnb_data["cleaning_fee"].fillna(0,inplace= True)

In [161]:
nan_values = airbnb_data.isnull().sum()
none_values = airbnb_data.isna().sum()
empty_string_values = (airbnb_data == '').sum()
missing_values = nan_values + none_values + empty_string_values
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
_id                    0
listing_url            0
name                   2
property_type          0
room_type              0
bed_type               0
minimum_nights         0
maximum_nights         0
cancellation_policy    0
first_review           0
last_review            0
accommodates           0
bedrooms               0
beds                   0
number_of_reviews      0
price                  0
security_deposit       0
cleaning_fee           0
guests_included        0
images                 0
review_scores          0
dtype: int64


In [162]:
airbnb_data.dtypes

_id                     object
listing_url             object
name                    object
property_type           object
room_type               object
bed_type                object
minimum_nights          object
maximum_nights          object
cancellation_policy     object
first_review            object
last_review             object
accommodates             int64
bedrooms               float64
beds                   float64
number_of_reviews        int64
price                   object
security_deposit        object
cleaning_fee            object
guests_included         object
images                  object
review_scores            int64
dtype: object

In [163]:
#changing_data_types:
airbnb_data["minimum_nights"]= airbnb_data["minimum_nights"].astype(int)
airbnb_data["maximum_nights"]= airbnb_data["maximum_nights"].astype(int)
airbnb_data["bedrooms"]= airbnb_data["bedrooms"].astype(int)
airbnb_data["beds"]= airbnb_data["beds"].astype(int)
airbnb_data["price"]= airbnb_data["price"].astype(str).astype(float).astype(int)
airbnb_data["security_deposit"]= airbnb_data["security_deposit"].astype(str).astype(float).astype(int)
airbnb_data["guests_included"]= airbnb_data["guests_included"].astype(str).astype(float).astype(int)
airbnb_data["cleaning_fee"]= airbnb_data["cleaning_fee"].astype(str).astype(float).astype(int)

In [164]:
airbnb_data.dtypes

_id                    object
listing_url            object
name                   object
property_type          object
room_type              object
bed_type               object
minimum_nights          int32
maximum_nights          int32
cancellation_policy    object
first_review           object
last_review            object
accommodates            int64
bedrooms                int32
beds                    int32
number_of_reviews       int64
price                   int32
security_deposit        int32
cleaning_fee            int32
guests_included         int32
images                 object
review_scores           int64
dtype: object

In [165]:
#HOST DETAILS

In [166]:
host_data = list(collection.find({}, {"_id": 1, "host": 1}))

host_columns = [{
    '_id': item['_id'],
    'host_id': item.get('host', {}).get('host_id'),
    'host_url': item.get('host', {}).get('host_url'),
    'host_name': item.get('host', {}).get('host_name'),
    'host_location': item.get('host', {}).get('host_location'),
    'host_response_time': item.get('host', {}).get('host_response_time'),
    'host_thumbnail_url': item.get('host', {}).get('host_thumbnail_url'),
    'host_picture_url': item.get('host', {}).get('host_picture_url'),
    'host_neighbourhood': item.get('host', {}).get('host_neighbourhood'),
    'host_response_rate': item.get('host', {}).get('host_response_rate'),
    'host_is_superhost': item.get('host', {}).get('host_is_superhost'),
    'host_has_profile_pic': item.get('host', {}).get('host_has_profile_pic'),
    'host_identity_verified': item.get('host', {}).get('host_identity_verified'),
    'host_listings_count': item.get('host', {}).get('host_listings_count'),
    'host_total_listings_count': item.get('host', {}).get('host_total_listings_count'),
    'host_verifications': item.get('host', {}).get('host_verifications')
} for item in host_data]
df_host = pd.DataFrame(host_columns)
df_host.head(5)

,_id,host_id,host_url,host_name,host_location,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,host_listings_count,host_total_listings_count,host_verifications
0,10006546,51399391,https://www.airbnb.com/users/show/51399391,Ana&Gonçalo,"Porto, Porto District, Portugal",within an hour,https://a0.muscache.com/im/pictures/fab79f25-2...,https://a0.muscache.com/im/pictures/fab79f25-2...,,100.0,False,True,True,3,3,"[email, phone, reviews, jumio, offline_governm..."
1,10009999,1282196,https://www.airbnb.com/users/show/1282196,Ynaie,"Rio de Janeiro, State of Rio de Janeiro, Brazil",None,https://a0.muscache.com/im/pictures/9681e3cc-4...,https://a0.muscache.com/im/pictures/9681e3cc-4...,Jardim Botânico,NaN,False,True,False,1,1,"[email, phone, facebook]"
2,1001265,5448114,https://www.airbnb.com/users/show/5448114,David,"Honolulu, Hawaii, United States",within an hour,https://a0.muscache.com/im/users/5448114/profi...,https://a0.muscache.com/im/users/5448114/profi...,Waikiki,98.0,False,True,False,18,18,"[email, phone, reviews, kba]"
3,10021707,11275734,https://www.airbnb.com/users/show/11275734,Josh,"New York, New York, United States",None,https://a0.muscache.com/im/users/11275734/prof...,https://a0.muscache.com/im/users/11275734/prof...,Bushwick,NaN,False,True,True,1,1,"[email, phone, reviews, kba]"
4,10030955,51496939,https://www.airbnb.com/users/show/51496939,Livia,BR,None,https://a0.muscache.com/im/pictures/b7911710-9...,https://a0.muscache.com/im/pictures/b7911710-9...,Lagoa,NaN,False,True,False,1,1,"[email, phone, jumio, government_id]"


In [167]:
nan_values = df_host.isnull().sum()
none_values = df_host.isna().sum()
empty_string_values = (df_host == '').sum()
missing_values = nan_values + none_values + empty_string_values
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
_id                             0
host_id                         0
host_url                        0
host_name                       0
host_location                   8
host_response_time           2776
host_thumbnail_url              0
host_picture_url                0
host_neighbourhood           1923
host_response_rate           2776
host_is_superhost               0
host_has_profile_pic            0
host_identity_verified          0
host_listings_count             0
host_total_listings_count       0
host_verifications              0
dtype: int64


In [168]:
df_host["host_response_time"].fillna(0, inplace=True)
df_host["host_neighbourhood"] = df_host["host_neighbourhood"].replace({'': "Not Specified"})
df_host["host_response_rate"].fillna(0, inplace=True)
df_host["host_location"] = df_host["host_location"].replace({'': "Not Specified"})

In [169]:
nan_values = df_host.isnull().sum()
none_values = df_host.isna().sum()
empty_string_values = (df_host == '').sum()
missing_values = nan_values + none_values + empty_string_values
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
_id                          0
host_id                      0
host_url                     0
host_name                    0
host_location                0
host_response_time           0
host_thumbnail_url           0
host_picture_url             0
host_neighbourhood           0
host_response_rate           0
host_is_superhost            0
host_has_profile_pic         0
host_identity_verified       0
host_listings_count          0
host_total_listings_count    0
host_verifications           0
dtype: int64


In [170]:
df_host["host_is_superhost"]= df_host["host_is_superhost"].map({False: "No", True: "Yes"})
df_host["host_has_profile_pic"]= df_host["host_has_profile_pic"].map({False: "No", True: "Yes"})
df_host["host_identity_verified"]= df_host["host_identity_verified"].map({False: "No", True: "Yes"})
df_host.head(5)

,_id,host_id,host_url,host_name,host_location,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,host_listings_count,host_total_listings_count,host_verifications
0,10006546,51399391,https://www.airbnb.com/users/show/51399391,Ana&Gonçalo,"Porto, Porto District, Portugal",within an hour,https://a0.muscache.com/im/pictures/fab79f25-2...,https://a0.muscache.com/im/pictures/fab79f25-2...,Not Specified,100.0,No,Yes,Yes,3,3,"[email, phone, reviews, jumio, offline_governm..."
1,10009999,1282196,https://www.airbnb.com/users/show/1282196,Ynaie,"Rio de Janeiro, State of Rio de Janeiro, Brazil",0,https://a0.muscache.com/im/pictures/9681e3cc-4...,https://a0.muscache.com/im/pictures/9681e3cc-4...,Jardim Botânico,0.0,No,Yes,No,1,1,"[email, phone, facebook]"
2,1001265,5448114,https://www.airbnb.com/users/show/5448114,David,"Honolulu, Hawaii, United States",within an hour,https://a0.muscache.com/im/users/5448114/profi...,https://a0.muscache.com/im/users/5448114/profi...,Waikiki,98.0,No,Yes,No,18,18,"[email, phone, reviews, kba]"
3,10021707,11275734,https://www.airbnb.com/users/show/11275734,Josh,"New York, New York, United States",0,https://a0.muscache.com/im/users/11275734/prof...,https://a0.muscache.com/im/users/11275734/prof...,Bushwick,0.0,No,Yes,Yes,1,1,"[email, phone, reviews, kba]"
4,10030955,51496939,https://www.airbnb.com/users/show/51496939,Livia,BR,0,https://a0.muscache.com/im/pictures/b7911710-9...,https://a0.muscache.com/im/pictures/b7911710-9...,Lagoa,0.0,No,Yes,No,1,1,"[email, phone, jumio, government_id]"


In [171]:
df_host.dtypes

_id                           object
host_id                       object
host_url                      object
host_name                     object
host_location                 object
host_response_time            object
host_thumbnail_url            object
host_picture_url              object
host_neighbourhood            object
host_response_rate           float64
host_is_superhost             object
host_has_profile_pic          object
host_identity_verified        object
host_listings_count            int64
host_total_listings_count      int64
host_verifications            object
dtype: object

In [172]:
#Location
location_data = list(collection.find({}, {"_id": 1, "address": 1}))

location_columns = [{
    '_id': item['_id'],
    'street': item.get('address', {}).get('street'),
    'suburb': item.get('address', {}).get('suburb'),
    'government_area': item.get('address', {}).get('government_area'),
    'market': item.get('address', {}).get('market'),
    'country': item.get('address', {}).get('country'),
    'country_code': item.get('address', {}).get('country_code'),
    'location_type': item.get('address', {}).get('location', {}).get('type'),
    'longitude': item.get('address', {}).get('location', {}).get('coordinates', [])[0],
    'latitude': item.get('address', {}).get('location', {}).get('coordinates', [])[1],
    'is_location_exact': item.get('address', {}).get('location', {}).get('is_location_exact')
} for item in location_data]
df_location = pd.DataFrame(location_columns)
df_location.head(5)

,_id,street,suburb,government_area,market,country,country_code,location_type,longitude,latitude,is_location_exact
0,10006546,"Porto, Porto, Portugal",,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",Porto,Portugal,PT,Point,-8.613080,41.141300,False
1,10009999,"Rio de Janeiro, Rio de Janeiro, Brazil",Jardim Botânico,Jardim Botânico,Rio De Janeiro,Brazil,BR,Point,-43.230750,-22.966254,True
2,1001265,"Honolulu, HI, United States",Oʻahu,Primary Urban Center,Oahu,United States,US,Point,-157.839190,21.286340,True
3,10021707,"Brooklyn, NY, United States",Brooklyn,Bushwick,New York,United States,US,Point,-73.936150,40.697910,True
4,10030955,"Rio de Janeiro, Rio de Janeiro, Brazil",Lagoa,Lagoa,Rio De Janeiro,Brazil,BR,Point,-43.205047,-22.971951,True


In [173]:
nan_values = df_location.isnull().sum()
none_values = df_location.isna().sum()
empty_string_values = (df_location == '').sum()
missing_values = nan_values + none_values + empty_string_values
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
_id                    0
street                 0
suburb               887
government_area        0
market                 6
country                0
country_code           0
location_type          0
longitude              0
latitude               0
is_location_exact      0
dtype: int64


In [174]:
df_location["suburb"] = df_location["suburb"].replace({'': "Not Specified"})
df_location["market"] = df_location["market"].replace({'': "Not Specified"})
df_location["is_location_exact"]= df_location["is_location_exact"].map({False: "No", True: "Yes"})
df_location.head(5)

,_id,street,suburb,government_area,market,country,country_code,location_type,longitude,latitude,is_location_exact
0,10006546,"Porto, Porto, Portugal",Not Specified,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",Porto,Portugal,PT,Point,-8.613080,41.141300,No
1,10009999,"Rio de Janeiro, Rio de Janeiro, Brazil",Jardim Botânico,Jardim Botânico,Rio De Janeiro,Brazil,BR,Point,-43.230750,-22.966254,Yes
2,1001265,"Honolulu, HI, United States",Oʻahu,Primary Urban Center,Oahu,United States,US,Point,-157.839190,21.286340,Yes
3,10021707,"Brooklyn, NY, United States",Brooklyn,Bushwick,New York,United States,US,Point,-73.936150,40.697910,Yes
4,10030955,"Rio de Janeiro, Rio de Janeiro, Brazil",Lagoa,Lagoa,Rio De Janeiro,Brazil,BR,Point,-43.205047,-22.971951,Yes


In [175]:
df_location.dtypes

_id                   object
street                object
suburb                object
government_area       object
market                object
country               object
country_code          object
location_type         object
longitude            float64
latitude             float64
is_location_exact     object
dtype: object

In [176]:
#Availiblity
availability_data = list(collection.find({}, {"_id": 1, "availability": 1}))

availability_columns = [{
    '_id': item['_id'],
    'availability_30': item.get('availability', {}).get('availability_30'),
    'availability_60': item.get('availability', {}).get('availability_60'),
    'availability_90': item.get('availability', {}).get('availability_90'),
    'availability_365': item.get('availability', {}).get('availability_365')
} for item in availability_data]

df_availability = pd.DataFrame(availability_columns)
df_availability.head(5)

,_id,availability_30,availability_60,availability_90,availability_365
0,10006546,28,47,74,239
1,10009999,0,0,0,0
2,1001265,16,46,76,343
3,10021707,0,0,0,0
4,10030955,28,58,88,363


In [177]:
nan_values = df_availability.isnull().sum()
none_values = df_availability.isna().sum()
empty_string_values = (df_availability == '').sum()
missing_values = nan_values + none_values + empty_string_values
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
_id                 0
availability_30     0
availability_60     0
availability_90     0
availability_365    0
dtype: int64


In [178]:
df_availability.dtypes

_id                 object
availability_30      int64
availability_60      int64
availability_90      int64
availability_365     int64
dtype: object

In [179]:
#Amenities
amenities_data = list(collection.find({}, {"_id": 1, "amenities": 1}))
df_amenities = pd.DataFrame(amenities_data)
df_amenities.head(5)

,_id,amenities
0,10006546,"[TV, Cable TV, Wifi, Kitchen, Paid parking off..."
1,10009999,"[Wifi, Wheelchair accessible, Kitchen, Free pa..."
2,1001265,"[TV, Cable TV, Wifi, Air conditioning, Pool, K..."
3,10021707,"[Internet, Wifi, Air conditioning, Kitchen, Bu..."
4,10030955,"[TV, Cable TV, Internet, Wifi, Air conditionin..."


In [180]:
nan_values = df_amenities.isnull().sum()
none_values = df_amenities.isna().sum()
empty_string_values = (df_amenities == '').sum()
missing_values = nan_values + none_values + empty_string_values
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
_id          0
amenities    0
dtype: int64


In [181]:
df_amenities.dtypes

_id          object
amenities    object
dtype: object

In [182]:
#Merging the All DataFrames
merged_df = pd.merge(airbnb_data, df_host, on="_id", suffixes=('_data', '_host'))
merged_df = pd.merge(merged_df, df_location, on="_id", suffixes=('_merged', '_location'))
merged_df = pd.merge(merged_df, df_availability, on="_id", suffixes=('_merged', '_availability'))
merged_df = pd.merge(merged_df, df_amenities, on="_id", suffixes=('_merged', '_amenities'))
merged_df.head(5)

,_id,listing_url,name,property_type,room_type,bed_type,minimum_nights,maximum_nights,cancellation_policy,first_review,...,country_code,location_type,longitude,latitude,is_location_exact,availability_30,availability_60,availability_90,availability_365,amenities
0,10006546,https://www.airbnb.com/rooms/10006546,Ribeira Charming Duplex,House,Entire home/apt,Real Bed,2,30,moderate,03-01-2016,...,PT,Point,-8.613080,41.141300,No,28,47,74,239,"[TV, Cable TV, Wifi, Kitchen, Paid parking off..."
1,1001265,https://www.airbnb.com/rooms/1001265,Ocean View Waikiki Marina w/prkg,Condominium,Entire home/apt,Real Bed,3,365,strict_14_with_grace_period,24-05-2013,...,US,Point,-157.839190,21.286340,Yes,16,46,76,343,"[TV, Cable TV, Wifi, Air conditioning, Pool, K..."
2,10021707,https://www.airbnb.com/rooms/10021707,Private Room in Bushwick,Apartment,Private room,Real Bed,14,1125,flexible,31-01-2016,...,US,Point,-73.936150,40.697910,Yes,0,0,0,0,"[Internet, Wifi, Air conditioning, Kitchen, Bu..."
3,1003530,https://www.airbnb.com/rooms/1003530,New York City - Upper West Side Apt,Apartment,Private room,Real Bed,12,360,strict_14_with_grace_period,29-04-2013,...,US,Point,-73.965230,40.799620,No,0,0,0,93,"[Internet, Wifi, Air conditioning, Kitchen, El..."
4,10038496,https://www.airbnb.com/rooms/10038496,Copacabana Apartment Posto 6,Apartment,Entire home/apt,Real Bed,3,75,strict_14_with_grace_period,18-01-2016,...,BR,Point,-43.190849,-22.984339,No,7,19,33,118,"[TV, Cable TV, Internet, Wifi, Air conditionin..."


In [183]:
file_path = r'C:\python\Airbnb_Analysis.csv'
merged_df.to_csv(file_path, index=False)